In [18]:
# Import the requests library.
import requests
from citipy import citipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import json as js

# Import the API key.
from config import api_key

In [2]:
beaches_df = pd.read_csv("Resources/ck_beaches.csv")
beaches_df.head()

,beach_id,beach_name,latitude,longitude
0,1,Pelican State Beach,41.992395,-124.209664
1,2,Crescent Beach,41.728363,-124.153178
2,3,Sue-meg State Park,41.136653,-124.155297
3,4,Trinidad State Beach,41.065822,-124.149789
4,5,Little River State Beach,41.016880,-124.111753


In [3]:
beaches_df.dtypes

beach_id        int64
beach_name     object
latitude      float64
longitude     float64
dtype: object

In [4]:
# Renamed "Unnamed: 0" column and set it as the index
beaches_df = beaches_df.rename(columns={'Unnamed: 0': ''})
beaches_df = beaches_df.set_index('beach_id')
beaches_df.head()

,beach_name,latitude,longitude
beach_id,,,
1,Pelican State Beach,41.992395,-124.209664
2,Crescent Beach,41.728363,-124.153178
3,Sue-meg State Park,41.136653,-124.155297
4,Trinidad State Beach,41.065822,-124.149789
5,Little River State Beach,41.016880,-124.111753


In [5]:
#beaches_df = beaches_df.drop_duplicates(keep='first')
#beaches_df

In [6]:
# Create the output file (CSV).
#output_data_file = "ck_beaches.csv"

# Export the beaches_df into a CSV.
#beaches_df.to_csv(output_data_file, index_label="beach_id")

In [7]:
# Create a set of latitude and longitude combinations.
beach = beaches_df['beach_name']
lats = beaches_df['latitude']
lngs = beaches_df['longitude']

beaches_lat_lngs = zip(beach, lats, lngs)
coordinates = list(beaches_lat_lngs)

for coordinate in coordinates:
    print(coordinate[0], coordinate[1], coordinate[2])

Pelican State Beach 41.992395 -124.209664
Crescent Beach 41.72836258 -124.1531784
Sue-meg State Park 41.13665293 -124.155297
Trinidad State Beach 41.06582175 -124.1497893
Little River State Beach 41.01688007 -124.1117529
Clam Beach County Park 40.99847401 -124.1151808
Clam Beach 40.99536694 -124.1109588
Sinkyone Wilderness State Park 39.92043479 -123.9397125
Westport-Union Landing State Beach 39.67716307 -123.7912683
MacKerricher Beach 39.489781 -123.798271
Glass Beach 39.45277476 -123.8133215
Caspar Beach 39.3627382 -123.8172252
Mendocino Headlands State Park 39.309033 -123.806658
Manchester State Park 38.98385949 -123.6993187
Schooner Gulch State Beach 38.86821949 -123.6547063
Gualala Point Regional Park 38.76076189 -123.526723
Black Point Beach 38.686409 -123.432347
Goat Rock Beach 38.44711794 -123.1265214
Blind Beach 38.43699972 -123.1223772
Shell Beach 38.417464 -123.105601
Wrights Beach 38.40033248 -123.0969266
Gleason Beach 38.39010789 -123.0857298
Schoolhouse Beach 38.37626954 

In [8]:
print(coordinates[193])

("Black's Beach", 32.89147257, -117.2559886)


In [9]:
len(beach)

205

In [10]:
# Starting URL for Weather Map API Call.
url = 'http://api.worldweatheronline.com/premium/v1/marine.ashx?key=' + api_key 
print(url)

http://api.worldweatheronline.com/premium/v1/marine.ashx?key=9242feb85e11428d81f20803220111


In [11]:
# Create an endpoint URL for a beach.
lat = 33.07705649
long = -117.3110183
beach_url = url + "&format=json&q=" + str(lat) + "," + str(long)
print(beach_url)

#api.openweathermap.org/data/2.5/forecast/daily?lat={lat}&lon={lon}&cnt={cnt}&appid={API key}

http://api.worldweatheronline.com/premium/v1/marine.ashx?key=9242feb85e11428d81f20803220111&format=json&q=33.07705649,-117.3110183


In [12]:
#for i, beach in enumerate(coordinates):
#    beach_url = url + "&format=json&q=" + str(coordinates[i-1][1]) + "," + str(coordinates[i-1][2])
#    beach_weather = requests.get(beach_url).json()
#    
#    sunrise = beaches_df['data']['weather'][0]['astronomy'][0]['sunrise']    
#    print(sunrise)

In [13]:
# test lat and long variables which are going to be beach coordinates
## using Grandview Coordinates= 33.07705649, -117.3110183
lat = 33.07705649
long = -117.3110183

# Create an endpoint URL for a beach. 

beach_url = url + "&format=json&q=" + str(lat) + "," + str(long)

#get it in text format
beach_weather = requests.get(beach_url)

if beach_weather.status_code == 200:
    print (f"Beach Weather found.")

else: 
    print (f"Beach weather not found.")

Beach weather not found.


In [14]:
# Create an empty list to hold the weather data.
beach_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the beaches in the list.
for i, j in enumerate(coordinates) :

    # Group beaches in sets of 30 for logging purposes.
    if (i % 30 == 0 and i >= 30):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each beach.
    beach_url = url + "&format=json&q=" + str(coordinates[i-1][1]) + "," + str(coordinates[i-1][2])

    # Log the URL, record, and set numbers and the beach.
    print(f"Processing Record {record_count} of Set {set_count} --- {coordinates[i-1][0]}")
    
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each beach.
    try:
        # Parse the JSON and retrieve data.
        beach_weather = requests.get(beach_url).json()
        
        # Parse out the needed data.
        sunrise = beach_weather['data']['weather'][0]['astronomy'][0]['sunrise']
        sunset = beach_weather['data']['weather'][0]['astronomy'][0]['sunset']
        weather = beach_weather['data']['weather'][0]['hourly'][2]['weatherDesc'][0]['value']
        temp = beach_weather['data']['weather'][0]['hourly'][2]['tempF']
        watertemp = beach_weather['data']['weather'][0]['hourly'][2]['waterTemp_F']
        cloudcover = beach_weather['data']['weather'][0]['hourly'][2]['cloudcover']
        windspeed = beach_weather['data']['weather'][0]['hourly'][2]['windspeedMiles']
        swell = beach_weather['data']['weather'][0]['hourly'][2]['swellHeight_ft']
        
        # Append the beach information into beach_data list.
        beach_data.append({"Beach_Name": coordinates [i-1][0],
                           "Latitude": coordinates [i-1][1],
                           "Longitude": coordinates [i-1][2],
                          "Sunrise": sunrise,
                          "Sunset": sunset,
                          "Weather_Description": weather,
                          "Temperature": temp,
                          "Water_Temperature": watertemp,
                          "Cloud_Cover": cloudcover,
                          "Wind_Speed": windspeed,
                          "Wave_Height": swell
                         })

# If an error is experienced, skip the beach.
    except:
        print("Beach information not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 --- Imperial Beach
Beach information not found. Skipping...
Processing Record 2 of Set 1 --- Pelican State Beach
Beach information not found. Skipping...
Processing Record 3 of Set 1 --- Crescent Beach
Beach information not found. Skipping...
Processing Record 4 of Set 1 --- Sue-meg State Park
Beach information not found. Skipping...
Processing Record 5 of Set 1 --- Trinidad State Beach
Beach information not found. Skipping...
Processing Record 6 of Set 1 --- Little River State Beach
Beach information not found. Skipping...
Processing Record 7 of Set 1 --- Clam Beach County Park
Beach information not found. Skipping...
Processing Record 8 of Set 1 --- Clam Beach
Beach information not found. Skipping...
Processing Record 9 of Set 1 --- Sinkyone Wilderness State Park
Beach information not found. Skipping...
Processing Record 10 of Set 1 --- Westport-Union Landing State Beach
Beach information not fou

Beach information not found. Skipping...
Processing Record 29 of Set 3 --- Shell Beach
Beach information not found. Skipping...
Processing Record 30 of Set 3 --- Pismo Beach Pier
Beach information not found. Skipping...
Processing Record 1 of Set 4 --- Rancho Guadalupe Dunes County Park
Beach information not found. Skipping...
Processing Record 2 of Set 4 --- Point Sal State Beach
Beach information not found. Skipping...
Processing Record 3 of Set 4 --- Surf Beach
Beach information not found. Skipping...
Processing Record 4 of Set 4 --- Jalama Beach County Park
Beach information not found. Skipping...
Processing Record 5 of Set 4 --- Gaviota State Park
Beach information not found. Skipping...
Processing Record 6 of Set 4 --- Refugio State Beach
Beach information not found. Skipping...
Processing Record 7 of Set 4 --- El Capitán State Beach
Beach information not found. Skipping...
Processing Record 8 of Set 4 --- Sands Beach
Beach information not found. Skipping...
Processing Record 9 o

Beach information not found. Skipping...
Processing Record 27 of Set 6 --- Carlsbad City Beach
Beach information not found. Skipping...
Processing Record 28 of Set 6 --- Carlsbad State Beach
Beach information not found. Skipping...
Processing Record 29 of Set 6 --- Terra Mar Point
Beach information not found. Skipping...
Processing Record 30 of Set 6 --- Ponto Beach
Beach information not found. Skipping...
Processing Record 1 of Set 7 --- South Carlsbad State Beach
Beach information not found. Skipping...
Processing Record 2 of Set 7 --- Grandview
Beach information not found. Skipping...
Processing Record 3 of Set 7 --- Beacon's Beach
Beach information not found. Skipping...
Processing Record 4 of Set 7 --- Stone Steps Beach
Beach information not found. Skipping...
Processing Record 5 of Set 7 --- Moonlight Beach
Beach information not found. Skipping...
Processing Record 6 of Set 7 --- D Street Viewpoint
Beach information not found. Skipping...
Processing Record 7 of Set 7 --- Swamis
B

In [15]:
#convert dictionaries into a Pandas DF
beach_data_df = pd.DataFrame(beach_data)
beach_data_df

""


In [16]:
# Create the output file (CSV).
#output_data_file = "beach_data_df.csv"

# Export the beach_data_df into a CSV.
#beach_data_df.to_csv(output_data_file, index_label="beach_id")

In [17]:
beach_data_df = pd.read_csv("beaches_data.csv")
beach_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'beaches_data.csv'

In [ ]:
# Renamed "Unnamed: 0" column and set it as the index
beach_data_df = beach_data_df.rename(columns={'Unnamed: 0': ''})
beach_data_df = beach_data_df.set_index('Beach_ID')
beach_data_df.head()

In [ ]:
wave_height_bins = [0, 3, 9, 12]
wave_size = ["Small (<4)", "Medium (4-9)", "Large (10-12)"]


# Categorize based on the bins.
beach_data_df["Wave_Size"] = pd.cut(beach_data_df["Wave_Height"], wave_height_bins, labels=wave_size)
beach_data_df

In [51]:
# Create the output file (CSV).
#output_data_file = "beach_data_df.csv"

# Export the beaches_data into a CSV.
#beach_data_df.to_csv(output_data_file, index_label="Beach_ID")